In [1]:
import pandas as pd
import numpy as np
import json
import recordlinkage

In [2]:
#import warnings
#warnings.filterwarnings('ignore')

In [3]:
with open('ep-popolo-v1.0.json', encoding="utf8") as f:
    json_dict = json.load(f)
    
df = pd.json_normalize(json_dict, record_path=['persons'])
df = df.rename(columns={'email':'primaryemail'})

In [4]:
df['contact_details'] = [ [{}] if x is np.NaN else x for x in df['contact_details'] ]

In [5]:
df_info = pd.json_normalize(df.to_dict('list'), ['contact_details']).unstack().apply(pd.Series)
df_extract = df_info.pivot_table(index=df_info.index.get_level_values(1), columns=['type'], 
                         values=['value'], aggfunc=','.join)
df = pd.concat([df, df_extract.xs('value', axis=1)], axis=1)

In [6]:
df = df[['birth_date','name','email','phone']]

In [7]:
df['year'] = pd.to_datetime(df['birth_date']).dt.year
df['month'] = pd.to_datetime(df['birth_date']).dt.month

In [8]:
df_psc = pd.read_csv('psc_slim.csv')

C:\Users\mshea\opencv\lib\site-packages\IPython\core\interactiveshell.py:3553: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
df_result = df.merge(df_psc, left_on=['name','year','month'], right_on=['data.name','data.date_of_birth.year','data.date_of_birth.month'],
          suffixes=('_left', '_right'))

In [10]:
df = df.rename({'name':'data.name'}, axis=1)

In [11]:
indexer = recordlinkage.Index()
indexer.block("data.name")
candidate_links = indexer.index(df, df_psc)
len(candidate_links)

1753

In [12]:
compare_cl = recordlinkage.Compare()
compare_cl.exact("data.name", "data.name", label="name")
compare_cl.exact("year", "data.date_of_birth.year", label="year_of_birth")
compare_cl.exact("month", "data.date_of_birth.month", label="month_of_birth")
features = compare_cl.compute(candidate_links, df, df_psc)

In [13]:
features

name  year_of_birth  month_of_birth
0    5883035     1              0               0
4    8091160     1              0               0
16   534711      1              0               0
     4397610     1              0               0
17   2535142     1              0               0
...            ...            ...             ...
1413 8839796     1              0               0
     8967232     1              0               0
1419 4619215     1              0               0
1425 4585761     1              0               0
1435 72824       1              0               0

[1753 rows x 3 columns]

In [14]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

3      26
2     153
1    1574
dtype: int64

In [15]:
features[features.sum(axis=1) > 2]

name  year_of_birth  month_of_birth
50   3936482     1              1               1
135  1593395     1              1               1
     1593770     1              1               1
     3734393     1              1               1
314  5929680     1              1               1
     6427452     1              1               1
     7418248     1              1               1
332  4622988     1              1               1
397  447090      1              1               1
432  6893411     1              1               1
479  3057967     1              1               1
494  610289      1              1               1
731  2150861     1              1               1
732  7265607     1              1               1
817  4820288     1              1               1
865  1500506     1              1               1
879  1978515     1              1               1
     5981342     1              1               1
     7065602     1              1               1
1089 7050571     1              1               1
     8215681     1              1               1
     8257703     1              1               1
1200 7700164     1              1               1
1287 65899       1              1               1
1412 1170431     1              1               1
     1197722     1              1               1